# Assignment 1

In [666]:
# #probl
# import sys
# !{sys.executable} -m pip install --user numpy --quiet
# !{sys.executable} -m pip install --user pandas --quiet
# !{sys.executable} -m pip install --user statsmodels --quiet
!{sys.executable} -m pip install --user scikit-learn --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [656]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

**Part A** 

In my process (below), data is generated for arrays Y1 and Y0 out of a normal distribution centered around 2 and 1, respectively. Because the Y1 values are all, or nearly all, greater than Y0, this introduces some bias. The two means estimates is not recovering the true ATE because of this "selection bias". Similarly, the confidence interval largely does not cover the true ATE because of this selection bias for which population to apply the treatment to.

A real world example of this is measuring the effect of people running everyday to their life expectancy. If the treatment group runs everyday day, there may be additional factors like diet or not smoking that would also affect life expectency.

In [688]:
def data_gen(n):
    p = 0.25
    # Z = np.random.normal(size = n, loc = 1)
    # generate average treatment array Y1
    Y1 =  np.random.normal(size=n, loc = 2, scale = 0.2)
    # generate average baseline array Y0
    Y0 = np.random.normal(size=n, loc = 1, scale = 0.2)
    # generating random binary treatment array with probability 0.25
    D = np.random.binomial(1, p, size=n)  
    # observed Y generated from [(treated * binary treatment array) + (baseline * inverse of binary tratment array)] + covariate
    Y = Y1 * 10 * D + Y0 * (1 - D)
    # demean covariates? 
    df = pd.DataFrame({"Y": Y, "D": D})
    return (df)




**Part B**


In [690]:
def estimation(n):
    df = data_gen(n)

    mean_treat = df[df['D'] == 1]['Y'].mean()
    mean_base = df[df['D'] == 0]['Y'].mean()
    ate = mean_treat - mean_base
                        
    var_treatment = df[df['D'] == 1]['Y'].var()
    var_base = var_treatment = df[df['D'] == 0]['Y'].var()

    v0 = var_base / (1 - df['D'].mean())
    v1 = var_treatment / df['D'].mean()
    se = np.sqrt((v0 + v1) / n)

    true_ate = 19

    return ate, true_ate, se

ate, true_ate, se = estimation(1000)
print(f"The two means estimate for n = 1000 is: {ate}")
print(f"The true ATE is {true_ate}")
print(f"The confidence interval is [{ate - 1.96 * se}, {ate + 1.96 * se}]")

The two means estimate for n = 1000 is: 19.137395468481937
The true ATE is 19
The confidence interval is [19.109124578617163, 19.165666358346712]


In [701]:
coverage_count = 0
running_bias = 0
sd_tme = []
for i in range(0, 100):
    ate, true_ate, se = estimation(1000)
    lower_bound = ate - 1.96 * se
    upper_bound = ate + 1.96 * se
    if (lower_bound < true_ate < upper_bound):
        coverage_count +=1
    running_bias += (ate - true_ate)
    sd_tme.append(ate)


print(coverage_count / 100) # Coverage
print(running_bias / 100) # Average bias
np.sqrt(np.var(sd_tme)) #Standard deviation of 2 means estimate



0.19
-0.005340311127482629


0.12618331051771953

# Assignment 2

**Part A**

In [657]:
#16-17
NV = 58;
NU = 61;
RV = 0/NV;
RU = 1/NU;
VE = (RU - RV)/RU;


tb = sm.stats.Table2x2([[0, NV - 0], [1, NU - 1]])
tb.summary()


print(f"Overall vaccine efficacy for 16-17 is {1 - tb.riskratio:.4f}")
lb, ub = tb.riskratio_confint(alpha=.05)
print(f"Approximate 95% confidence interval of VE for 16-17: [{1 - ub:.4f}, {1 - lb:.4f}]")

#18-64
NV = 14443;
NU = 14566;
RV = 8/NV;
RU = 149/NU;
VE = (RU - RV)/RU;


tb = sm.stats.Table2x2([[8, NV - 8], [149, NU - 149]])
tb.summary()


print(f"Overall vaccine efficacy for 18-64 is {1 - tb.riskratio:.4f}")
lb, ub = tb.riskratio_confint(alpha=.05)
print(f"Approximate 95% confidence interval of VE for 18-64: [{1 - ub:.4f}, {1 - lb:.4f}]")

Overall vaccine efficacy for 16-17 is 0.4786
Approximate 95% confidence interval of VE for 16-17: [-14.2481, 0.9822]
Overall vaccine efficacy for 18-64 is 0.9459
Approximate 95% confidence interval of VE for 18-64: [0.8898, 0.9734]


The efficacy for 16-17 is not recovered (0.48 vs the reported 1.00), likely due to the relatively low number of samples. However, the 18-64 group in the code is 0.9459 compared to the CDC's 0.946.

**Part B**

In [658]:
#16-17
NV = 58;
NU = 61;
RV = 0/NV; #Expectation of Vaccinated Group (Theta_1)
RU = 1/NU; #Expectation of Unvaccinated Group (Theta_0)
VE = (RU - RV)/RU;


V11 = RU * (1 - RU) / NU #Variance of Unvaccinated Group
V22 = RV * (1- RV) / NV #Variance of Vaccinated Group


RV_sqrd = RV**2;
RU_sqrd = RU**2;
RU_frth = RU**4;


delta_var = (RV_sqrd * V11)/RU_frth + (V22/RU_sqrd) #Variance via the delta method
delta_se = np.sqrt(delta_var) #Standard Error via the delta method


delta_ci = [VE - 1.96*delta_se, VE + 1.96*delta_se]


print(f"Vaccine Efficacy estimate is {VE:.4f}")
print(f"95% confidence interval using the delta method is [{delta_ci[0]:.4f}, {delta_ci[1]:.4f}]")
###18-64
NV = 14443
NU = 14566
RV = 8/NV; #Expectation of Vaccinated Group (Theta_1) 
RU = 149/NU; #Expectation of Unvaccinated Group (Theta_0) 
VE = (RU - RV)/RU;


V11 = RU * (1 - RU) / NU #Variance of Unvaccinated Group 
V22 = RV * (1- RV) / NV #Variance of Vaccinated Group 
RV_sqrd = RV**2; 
RU_sqrd = RU**2; 
RU_frth = RU**4; 


delta_var = (RV_sqrd * V11)/RU_frth + (V22/RU_sqrd) #Variance via the delta method
delta_var = (mean_con**2 * var_base)/mean_base**4 + (var_treatment/mean_base**2)
delta_se = np.sqrt(delta_var) #Standard Error via the delta method


delta_ci = [VE - 1.96*delta_se, VE + 1.96*delta_se]


print(f"Vaccine Efficacy estimate is {VE:.4f}")
print(f"95% confidence interval using the delta method is [{delta_ci[0]:.4f}, {delta_ci[1]:.4f}]")

Vaccine Efficacy estimate is 1.0000
95% confidence interval using the delta method is [1.0000, 1.0000]
Vaccine Efficacy estimate is 0.9459
95% confidence interval using the delta method is [-40.4607, 42.3524]


The delta method was implemented from the formula on lecture 2 slide 23. The formula came from the normal distribution where the mean is 0 and the variance is given by the sum of the two fractions. The first fraction is the product of the vaccinated probability squared times the variance of the placebo group and then divided by the placebo probability to the power of four. That fraction is added to the variance of vaccinated divided by placebo probability of infection squared.

The standard error is the square root of that given variance described above.

# Problem 3


In [686]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

data = pd.read_csv("https://raw.githubusercontent.com/VC2015/DMLonGitHub/master/penn_jae.dat", delim_whitespace=True)
n, p = data.shape
data = data[data["tg"].isin([0, 4])]

data["T4"] = np.where(data["tg"] == 4, 1, 0)
data["T4"].value_counts()

covariate_balance = smf.ols("T4~(female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2",
                           data)
covariate_balance.fit(cov_type="HC1").summary()

# no adjustment (2-sample approach)
cl = smf.ols("np.log(inuidur1) ~ T4", data)


# adding controls
cra = smf.ols("np.log(inuidur1) ~ T4 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2",
             data)
# Omitted dummies: q1, nondurable, muld


cl_results = cl.fit(cov_type="HC1")
cl_results.summary()

cra_results = cra.fit(cov_type="HC1")
cra_results.summary()


#interactive regression model

ira_formula = "0 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2"
X = patsy.dmatrix(ira_formula, data, return_type='dataframe')
X.columns = [f'x{t}' for t in range(X.shape[1])] # clean column names
X = (X - X.mean(axis=0)) # demean all control covariates

# construct interactions of treatment and (de-meaned covariates, 1)
ira_formula = "T4 * ("+ "+".join(X.columns) + ")"
X['T4'] = data['T4']
X = patsy.dmatrix(ira_formula, X, return_type='dataframe')
ira = sm.OLS(np.log(data[["inuidur1"]]), X)
ira_results = ira.fit(cov_type="HC1")
ira_results.summary()


from sklearn.linear_model import LassoCV


D = X['T4']
A = X.drop('T4', axis=1) # contains X and interaction terms D*X
y = np.log(data["inuidur1"])


Dres = D - np.mean(D)
yres = y - LassoCV(max_iter=10000).fit(A, y).predict(A)


ira_lasso_results = sm.OLS(yres, Dres).fit(cov_type='HC1')
ira_lasso_results.summary()

results = {
    "CL": [cl_results.params[cl.exog_names.index("T4")],
           cl_results.HC1_se[cl.exog_names.index("T4")]],
    "CRA": [cra_results.params[cra.exog_names.index("T4")],
           cra_results.HC1_se[cra.exog_names.index("T4")]],
    "IRA": [ira_results.params[ira.exog_names.index("T4")],
           ira_results.HC1_se[ira.exog_names.index("T4")]],
    "IRA w Lasso": [ira_lasso_results.params[0],
           ira_lasso_results.bse[0]],
}
results = pd.DataFrame(results)
results.index = ["Estimate", "Standard Error"]
results
# for printing to LaTeX: print(results.to_latex())


C:\Users\nthnj\AppData\Local\Temp\ipykernel_38412\3346323124.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv("https://raw.githubusercontent.com/VC2015/DMLonGitHub/master/penn_jae.dat", delim_whitespace=True)
C:\Users\nthnj\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 119, but rank is 102
  warnings.warn('covariance of constraints does not have full '
C:\Users\nthnj\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 120, but rank is 103
  warnings.warn('covariance of constraints does not have full '
C:\Users\nthnj\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of co

,CL,CRA,IRA,IRA w Lasso
Estimate,-0.085455,-0.079680,347276.374040,-0.078782
Standard Error,0.035856,0.035591,359569.173314,0.034886


Note* I have no idea why my IRA results came out like this. I copied the cells exactly multiple times. At least its consistent across the board.

In [671]:
covariate_balance = smf.ols("T4~(female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2 + (female+black+othrace+agelt35+agegt54)**3",
                           data)
### Three way interactions: (female+black+othrace+agelt35+agegt54)**3


covariate_balance.fit(cov_type="HC1").summary()

cl_new = smf.ols("np.log(inuidur1) ~ T4", data)


# adding controls
cra_new = smf.ols("np.log(inuidur1) ~ T4 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2 + (female+black+othrace+agelt35+agegt54)**3",
             data)
# Omitted dummies: q1, nondurable, muld


cl_new_results = cl_new.fit(cov_type="HC1")
cl_new_results.summary()

cra_new_results = cra_new.fit(cov_type="HC1")
cra_new_results.summary()

#interactive regression model


ira_formula_new = "0 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2 + (female+black+othrace+agelt35+agegt54)**3"
X = patsy.dmatrix(ira_formula_new, data, return_type='dataframe')
X.columns = [f'x{t}' for t in range(X.shape[1])] # clean column names
X = (X - X.mean(axis=0)) # demean all control covariates


# construct interactions of treatment and (de-meaned covariates, 1)
ira_formula_new = "T4 * ("+ "+".join(X.columns) + ")"
X['T4'] = data['T4']
X = patsy.dmatrix(ira_formula_new, X, return_type='dataframe')
ira_new = sm.OLS(np.log(data[["inuidur1"]]), X)
ira_new_results = ira_new.fit(cov_type="HC1")
ira_new_results.summary()

D = X['T4']
A = X.drop('T4', axis=1) # contains X and interaction terms D*X
y = np.log(data["inuidur1"])


Dres = D - np.mean(D)
yres = y - LassoCV(max_iter=10000).fit(A, y).predict(A)


ira_lasso_new_results = sm.OLS(yres, Dres).fit(cov_type='HC1')
ira_lasso_new_results.summary()
results_new = {
   "CL_New": [cl_new_results.params[cl_new.exog_names.index("T4")],
          cl_new_results.HC1_se[cl_new.exog_names.index("T4")]],
   "CRA_New": [cra_new_results.params[cra_new.exog_names.index("T4")],
          cra_new_results.HC1_se[cra_new.exog_names.index("T4")]],
   "IRA_New": [ira_new_results.params[ira_new.exog_names.index("T4")],
          ira_new_results.HC1_se[ira_new.exog_names.index("T4")]],
   "IRA w Lasso New": [ira_lasso_new_results.params[0],
          ira_lasso_new_results.bse[0]],
}
results_new = pd.DataFrame(results_new)
results_new.index = ["Estimate", "Standard Error"]
results_new
# for printing to LaTeX: print(results.to_latex())

###Original Model Results
results



C:\Users\nthnj\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 129, but rank is 106
  warnings.warn('covariance of constraints does not have full '
C:\Users\nthnj\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 130, but rank is 107
  warnings.warn('covariance of constraints does not have full '
C:\Users\nthnj\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 261, but rank is 1
  warnings.warn('covariance of constraints does not have full '
C:\Users\nthnj\AppData\Local\Temp\ipykernel_38412\2993639710.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will 

,CL,CRA,IRA,IRA w Lasso
Estimate,-0.085455,-0.079680,347276.374040,-0.078782
Standard Error,0.035856,0.035591,359569.173314,0.034886


**Part B**

In [677]:
covariate_balance = smf.ols("T4~(female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2 + (female+black+othrace+agelt35+agegt54)**3",
                           data)
### Three way interactions: (female+black+othrace+agelt35+agegt54)**3


covariate_balance.fit().summary() #cov_type=nonrobust

# no adjustment (2-sample approach)
cl_new_nr = smf.ols("np.log(inuidur1) ~ T4", data)


# adding controls
cra_new_nr = smf.ols("np.log(inuidur1) ~ T4 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2 + (female+black+othrace+agelt35+agegt54)**3",
             data)
# Omitted dummies: q1, nondurable, muld


cl_new_nr_results = cl_new_nr.fit() #cov_type=nonrobust
cl_new_nr_results.summary()

cra_new_nr_results = cra_new_nr.fit() #cov_type=nonrobust
cra_new_nr_results.summary()

#interactive regression model


ira_formula_new = "0 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2 + (female+black+othrace+agelt35+agegt54)**3"
X = patsy.dmatrix(ira_formula_new, data, return_type='dataframe')
X.columns = [f'x{t}' for t in range(X.shape[1])] # clean column names
X = (X - X.mean(axis=0)) # demean all control covariates


# construct interactions of treatment and (de-meaned covariates, 1)
ira_formula_new = "T4 * ("+ "+".join(X.columns) + ")"
X['T4'] = data['T4']
X = patsy.dmatrix(ira_formula_new, X, return_type='dataframe')
ira_new_nr = sm.OLS(np.log(data[["inuidur1"]]), X)
ira_new_nr_results = ira_new_nr.fit() #cov_type=nonrobust
ira_new_nr_results.summary()

D = X['T4']
A = X.drop('T4', axis=1) # contains X and interaction terms D*X
y = np.log(data["inuidur1"])


Dres = D - np.mean(D)
yres = y - LassoCV(max_iter=10000).fit(A, y).predict(A)


ira_lasso_new_nr_results = sm.OLS(yres, Dres).fit() #cov_type=None
ira_lasso_new_nr_results.summary()

results_new_nr = {
   "CL_New_Non_Robust_SE": [cl_new_nr_results.params[cl_new_nr.exog_names.index("T4")],
          cl_new_nr_results.bse[cl_new_nr.exog_names.index("T4")]],
   "CRA_New_Non_Robust_SE": [cra_new_nr_results.params[cra_new_nr.exog_names.index("T4")],
          cra_new_nr_results.bse[cra_new_nr.exog_names.index("T4")]],
   "IRA_New_Non_Robust_SE": [ira_new_nr_results.params[ira_new_nr.exog_names.index("T4")],
          ira_new_nr_results.bse[ira_new_nr.exog_names.index("T4")]],
   "IRA w Lasso New_Non_Robust_SE": [ira_lasso_new_nr_results.params[0],
          ira_lasso_new_nr_results.bse[0]],
}
results_new_nr = pd.DataFrame(results_new_nr)
results_new_nr.index = ["Estimate", "Non-Robust Standard Error"]
results_new_nr
# for printing to LaTeX: print(results.to_latex())

###New Model Results w/ cov_type=HC1
results_new


C:\Users\nthnj\AppData\Local\Temp\ipykernel_38412\3657216273.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "CL_New_Non_Robust_SE": [cl_new_nr_results.params[cl_new_nr.exog_names.index("T4")],
C:\Users\nthnj\AppData\Local\Temp\ipykernel_38412\3657216273.py:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cl_new_nr_results.bse[cl_new_nr.exog_names.index("T4")]],
C:\Users\nthnj\AppData\Local\Temp\ipykernel_38412\3657216273.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

,CL_New,CRA_New,IRA_New,IRA w Lasso New
Estimate,-0.085455,-0.078807,-337259.281584,-0.078824
Standard Error,0.035856,0.035617,348599.847134,0.034910


The CRA and IRA w/Lasso standard error are the only difference errors and they are relatively negligible in their differences. Because the robust and non-robust errors are essentially the same, we can assume the data is homoskedastic in the set. 

# Problem 4


**Part A**

|          |      |
|----------|------|
| **CL**   | 0.106|
| **CRA**  | 0.138|
| **IRA**  | 0.08 |


Adjusting for covariates without interaction terms reduced precision (as seen with the increase in SE under CL v. CRA) whereas adjusting for covariates with interaction terms did improve precision (as seen with the change for IRA v. CRA)

**Part B**
|          |      |
|----------|------|
| CL  | 0.113 |
| CRA | 0.098 |
| IRA | 0.078 |



The difference arrizes from the np.random.normal function in the data generation process, which takes from the normal distribution to create y0, y1, and the covariate z. Because z adds or subtracts from the baseline or treatment, it creates heteroskedacisty in the set due to the sums of the outlying values bringing some variables further outside of the normal distribution. This would increase the variation in the values and thus the heteroskedacisty.



**Part C**

|          |  Precision Adjustment  |   Non-Robust  |
|----------|------|------|
| CL  | 0.087 | 0.087  |
| CRA | 0.087 | 0.087   |
| IRA | 0.061 | 0.061|




Adjusting the probability to 0.5 made the data homoskedastic, therefore adjusting between robust and non-robust makes no difference.

**Part D**
Below